In [1]:
from spatialvla.datasets.rlds.utils.data_utils import load_statistics_from_json

2024-11-05 22:36:39.722146: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-05 22:36:39.751562: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 22:36:39.751609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 22:36:39.752515: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 22:36:39.757809: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from spatialvla.datasets import RLDSBatchTransform, RLDSDataset

In [2]:
load_statistics_from_json('/checkpoints')

FileNotFoundError: [Errno 2] No such file or directory: '/checkpoints/dataset_statistics.json'

In [1]:
from spatialvla.mobilevlm.model.mobilevlm import load_pretrained_vlm_for_vla
from scripts.spatialvla_config_revised import ModelArguments, TrainingArguments
import transformers

model_args = ModelArguments()

tokenizer, model, image_processor, _ = load_pretrained_vlm_for_vla(
    model_args,
    load_8bit=False, 
    load_4bit=False,
    device_map='auto',
    device='cuda',
)

[2024-11-05 20:48:47,400] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2024-11-05 20:48:47.711408: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-05 20:48:47.740083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 20:48:47.740130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 20:48:47.741012: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 20:48:47.746229: I tensorflow/core/platform/cpu_feature_guar

In [2]:
model

SpatialVLAForCausalLM(
  (model): SpatialVLAModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [4]:
## RLDS Dataset loading
batch_transform = RLDSBatchTransform(
        tokenizer,
        image_processor,
    )
# Init complete

In [5]:
cfg = TrainingArguments()
dataset_kwargs

NameError: name 'dataset_kwargs' is not defined

In [5]:
vla_dataset = RLDSDataset(
        data_root_dir=],
        data_mix='lg_mix',
        batch_transform=batch_transform,
        shuffle_buffer_size=100,
        window_size=1,
        future_action_window_size=4
    )

2024-11-05 17:01:07.997759: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-11-05 17:01:08.396843: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-11-05 17:01:08.776810: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-11-05 17:01:08.878704: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization



######################################################################################
# Loading the following 2 datasets (incl. sampling weight):                         #
# lg_cup_color_rightarm: ===================================================0.424252 #
# lg_stack_cup_5hz: ========================================================0.575748 #
######################################################################################

Threads per Dataset: %s [1 1]
Reads per Dataset: %s [1 1]
Constructing datasets...


2024-11-05 17:01:09.006076: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-11-05 17:01:09.777867: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


Applying frame transforms on dataset...
we are decodign here
we are decodign here


In [6]:
from spatialvla.datasets.rlds.utils.data_utils import PaddedCollatorForActionPrediction

collator = PaddedCollatorForActionPrediction(tokenizer.model_max_length, tokenizer.pad_token_id, padding_side='right')

In [7]:

from torch.utils.data import DataLoader
dataloader = DataLoader(
        vla_dataset,
        batch_size=32,
        sampler=None,
        collate_fn=collator,
        num_workers=0,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
    )

In [8]:
b = None
for batch in dataloader:
    b = batch
    break

/home/jellyho/Bimanual_Imitation/MobileVLM-VLA/spatialvla/datasets/datasets.py:50: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  dataset_name, action = rlds_batch["dataset_name"], torch.Tensor(rlds_batch["action"]).to(torch.float16)


In [9]:
b['pixel_values'].shape

torch.Size([32, 1, 3, 336, 336])

In [10]:
b['action'].shape

torch.Size([32, 5, 7])

In [11]:
b['input_ids'].shape

torch.Size([32, 62])